# **Feature Engineering**

## **1 Import Libraries**





In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [ ]:
!pip install stop-words

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import time
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import gc
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import math
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestRegressor
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from prettytable import PrettyTable
import pickle
import zipfile
from tqdm import tqdm
import os
from sklearn.linear_model import Ridge

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from stop_words import get_stop_words
from collections import Counter

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense,Flatten
from tensorflow.keras.models import Model
from numpy import zeros
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
tf.keras.backend.clear_session()

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### **1.1 import data**



In [ ]:
PATH = 'My Drive/kì 2/học máy/mercari-price-suggestion-challenge/'

In [ ]:
train = pd.read_csv('/content/gdrive/'+ PATH +'data/train.tsv',sep='\t')

In [ ]:
test = pd.read_csv('/content/gdrive/'+ PATH + 'data/test_stg2.tsv',sep='\t')

In [ ]:
train.shape

(1482535, 8)

In [ ]:
test.shape

(3460725, 7)

In [ ]:
print(train.columns)

Index(['train_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'price', 'shipping', 'item_description'],
      dtype='object')


In [ ]:
print(test.columns)

Index(['test_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'shipping', 'item_description'],
      dtype='object')


In [ ]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   train_id           1482535 non-null  int64  
 1   name               1482535 non-null  object 
 2   item_condition_id  1482535 non-null  int64  
 3   category_name      1476208 non-null  object 
 4   brand_name         849853 non-null   object 
 5   price              1482535 non-null  float64
 6   shipping           1482535 non-null  int64  
 7   item_description   1482531 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 90.5+ MB


In [ ]:
print(train.isnull().sum())

train_id                  0
name                      0
item_condition_id         0
category_name          6327
brand_name           632682
price                     0
shipping                  0
item_description          4
dtype: int64


**Splitting categories into subcategories:**

In [ ]:
# tách caterory ra làm 3 category con
def split_categories(category):
  try:
    sub_cat_1,sub_cat_2,sub_cat_3 = category.split("/")
    return sub_cat_1,sub_cat_2,sub_cat_3
  except:
    return ("No label","No label","No label")

In [ ]:
# sử dụng hàm split_categories để tách category ra làm 3 category con và tạo thành 3 cột mới : 'main_category', 'sub_cat_1','sub_cat_2'
def create_split_categories(data):
  data['main_category'],data['sub_cat_1'],data['sub_cat_2']=zip(*data['category_name'].\
                                                                apply(lambda x: split_categories(x)))

In [ ]:
create_split_categories(train)
create_split_categories(test)

In [ ]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,main_category,sub_cat_1,sub_cat_2
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,Men,Tops,T-shirts
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces


**Chuyển đổi giá sang log**

In [ ]:
def log_price(price):
    return np.log1p(price)

In [ ]:
train['log_price'] = log_price(train['price'])

**Thêm trường số lượng từ trong phần mô tả**

In [ ]:
train['description_wc'] = [len(str(i).split()) for i in train['item_description']]

## **2 Data Preprocessing**

### **2.1 Data cleaning:**

Loại bỏ các sản phẩm có giá thành nhỏ hơn 3$ vì giá tối thiểu là 3 trên trang web

In [ ]:
train = train.drop(train[(train.price < 3.0)].index)

In [ ]:
print(train.shape)

(1481661, 13)


### **2.2 Handling Null/Missing values:**

In [ ]:
print(train.isnull().sum())

train_id                  0
name                      0
item_condition_id         0
category_name          6314
brand_name           632336
price                     0
shipping                  0
item_description          4
main_category             0
sub_cat_1                 0
sub_cat_2                 0
log_price                 0
description_wc            0
dtype: int64


Với các trường brand_name và category_name thì thay thế NaN bằng 'missing', với trường item_description thì thay thế NaN bằng 'No description yet'

In [ ]:
def fill_nan(dataset):
  dataset["item_description"].fillna("No description yet",inplace=True)
  dataset["brand_name"].fillna("missing",inplace=True)
  dataset["category_name"].fillna("missing",inplace=True)

In [ ]:
fill_nan(train)
fill_nan(test)

In [ ]:
print(train.isnull().sum())

train_id             0
name                 0
item_condition_id    0
category_name        0
brand_name           0
price                0
shipping             0
item_description     0
main_category        0
sub_cat_1            0
sub_cat_2            0
log_price            0
description_wc       0
dtype: int64


In [ ]:
print(test.isnull().sum())

test_id              0
name                 0
item_condition_id    0
category_name        0
brand_name           0
shipping             0
item_description     0
main_category        0
sub_cat_1            0
sub_cat_2            0
dtype: int64


### **2.3 Insert feature**

Tạo thêm 2 cột wc_desc và wc_name dựa vào độ dài từ của trường item_description và name

In [ ]:
def countwords(text):
    try:
        if text == 'No description yet':
            return 0
        else:
            text = text.lower()
            words = [w for w in text.split(" ")]
            return len(words)
    except: 
        return 0

In [ ]:
train['wc_desc'] = train['item_description'].apply(lambda x: countwords(x))
train['wc_name'] = train['name'].apply(lambda x: countwords(x))

In [ ]:
train.columns

Index(['train_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'price', 'shipping', 'item_description', 'main_category', 'sub_cat_1',
       'sub_cat_2', 'log_price', 'description_wc', 'wc_desc', 'wc_name'],
      dtype='object')

In [ ]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,main_category,sub_cat_1,sub_cat_2,log_price,description_wc,wc_desc,wc_name
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,missing,10.0,1,No description yet,Men,Tops,T-shirts,2.397895,3,0,7
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,3.970292,36,36,4
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,2.397895,29,29,2
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,missing,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,3.583519,32,32,3
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,missing,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,3.806662,5,5,4


Create pickle dumps for train and test:

In [ ]:
train.to_pickle('/content/gdrive/'+ PATH + 'pickles/train.pkl')
test.to_pickle('/content/gdrive/'+ PATH + 'pickles/test.pkl')

In [ ]:
train = pd.read_pickle('/content/gdrive/'+ PATH + 'pickles/train.pkl')
test = pd.read_pickle('/content/gdrive/'+ PATH + 'pickles/test.pkl')

## **3 Feature Construction**

**TFIDF** cho trường item_description sử dụng uni,bi-grams , **Bag of Words** cho trường name sử dụng uni- và bi-grams, **one hot encodes** cho tất cả các trường có tính phân loại còn lại

**Train-Test split from train dataset**

Ta chia tập train ra làm 2 phần theo tỉ lệ **1/9** , 1 phần cho việc huấn luyện, 1 phần cho việc kiểm thử

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test= train_test_split(train, test_size=0.10, random_state=123)

In [ ]:
X_train.to_pickle('/content/gdrive/'+ PATH + 'pickles/X_train.pkl')
X_test.to_pickle('/content/gdrive/'+ PATH + 'pickles/X_test.pkl')

In [ ]:
X_train = pd.read_pickle('/content/gdrive/'+ PATH + 'pickles/X_train.pkl')
X_test = pd.read_pickle('/content/gdrive/'+ PATH + 'pickles/X_test.pkl')

In [ ]:
X_train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,main_category,sub_cat_1,sub_cat_2,log_price,description_wc,wc_desc,wc_name
1191489,1191489,Katy Perry,1,Beauty/Fragrance/Women,CoverGirl,6.0,1,No description yet,Beauty,Fragrance,Women,1.945910,3,0,2
247715,247715,Tarte foundation,1,Beauty/Makeup/Face,Tarte,25.0,1,"New, light neutral color, 1.7oz",Beauty,Makeup,Face,3.258097,5,5,2
821140,821140,Bumble and bumble Hair Bundle,2,Beauty/Hair Care/Styling Products,Sephora,36.0,1,"Brand new, unused multitasking Hairdressers In...",Beauty,Hair Care,Styling Products,3.610918,52,52,5
782968,782968,Bath and Body Works,2,Beauty/Bath & Body/Sets,Bath & Body Works,9.0,0,Bath & Body Works Cashmere Glow Golden Sugar S...,Beauty,Bath & Body,Sets,2.302585,12,12,4
1370038,1370038,Victoria's secret body by Victoria bra,3,Women/Underwear/Bras,Victoria's Secret,14.0,1,38B Gray Push-up Good condition My reference c...,Women,Underwear,Bras,2.708050,9,9,6


### **3.1 Encoding of categorical feature:**

Các features có tính phân loại như **main_category,sub_cat_1,sub_cat_2,brand_name** sẽ sử dụng **one hot encoded**. **shipping_id** và **item_condition_id** sẽ được encoded bằng cách sử dụng hàm **get_dummies()**

In [ ]:
def one_hot_encode(train,test):
    '''
    Function to one hot encode the categorical columns
    '''
    vectorizer = CountVectorizer(token_pattern='.+')
    # ta chỉ sử dụng hàm fit trên tậm dữ liệu train

    vectorizer = vectorizer.fit(train['category_name'].values)
    column_cat = vectorizer.transform(test['category_name'].values)

    vectorizer = vectorizer.fit(train['main_category'].values)
    column_mc = vectorizer.transform(test['main_category'].values)
    
    vectorizer = vectorizer.fit(train['sub_cat_1'].values)
    column_sb1 = vectorizer.transform(test['sub_cat_1'].values)
    
    vectorizer = vectorizer.fit(train['sub_cat_2'].values)
    column_sb2 = vectorizer.transform(test['sub_cat_2'].values)

    vectorizer = vectorizer.fit(train['brand_name'].astype(str))
    brand_encodes = vectorizer.transform(test['brand_name'].astype(str))
   
    print(column_cat.shape)
    print(column_mc.shape)
    print(column_sb1.shape)
    print(column_sb2.shape)
    print(brand_encodes.shape)
    print("="*100)
    return column_cat,column_mc,column_sb1,column_sb2,brand_encodes

One hot encodings cho **category, main_category, sub_cat_1,sub_cat_2, brand_name** trong tập **X_train**:


In [ ]:
cat_ohe_tr,main_cat_ohe_tr,sub_cat_1_ohe_tr,sub_cat_2_ohe_tr,brand_ohe_tr = one_hot_encode(X_train,X_train)

(1333494, 1276)
(1333494, 11)
(1333494, 114)
(1333494, 858)
(1333494, 4681)


One hot encodings cho **category, main_category, sub_cat_1,sub_cat_2,brand_name** trong tập **X_test**:

In [ ]:
cat_ohe_te,main_cat_ohe_te,sub_cat_1_ohe_te,sub_cat_2_ohe_te,brand_ohe_te = one_hot_encode(X_train,X_test)

(148167, 1276)
(148167, 11)
(148167, 114)
(148167, 858)
(148167, 4681)


In [ ]:
def get_dummies_item_id_shipping(df):
    df['item_condition_id'] = df["item_condition_id"].astype("category")
    df['shipping'] = df["shipping"].astype("category")
    item_id_shipping = csr_matrix(pd.get_dummies(df[['item_condition_id', 'shipping']],\
                                                      sparse=True).values)
    return item_id_shipping

In [ ]:
item_id_shipping_X_tr = get_dummies_item_id_shipping(X_train)
item_id_shipping_X_te = get_dummies_item_id_shipping(X_test)

Hợp nhất các features vào một ma trận sử dụng hàm hstack()

In [ ]:
tr_cat = hstack((cat_ohe_tr,main_cat_ohe_tr,sub_cat_1_ohe_tr,sub_cat_2_ohe_tr,brand_ohe_tr,\
                     item_id_shipping_X_tr)).tocsr()
te_cat = hstack((cat_ohe_te,main_cat_ohe_te,sub_cat_1_ohe_te,sub_cat_2_ohe_te,brand_ohe_te,\
                     item_id_shipping_X_te)).tocsr()

print(tr_cat.shape)
print(te_cat.shape)

(1333494, 6947)
(148167, 6947)


In [ ]:
def pickle_dump(*args,filename = 'file'):
    argv = args
    list_variable = [i for i in argv]
    dir_name = "/content/gdrive/"+ PATH + "pickles/"
    filename_suffix = "pkl"
    file_path = os.path.join(dir_name, filename + "." + filename_suffix)
    with open(file_path, 'wb') as f:
        pickle.dump(list_variable, f)

In [ ]:
pickle_dump(tr_cat,te_cat,filename = "cat_features")

In [ ]:
def read_pickle_dump(*args,filename = 'file'):
    argv = args
    variable_list = [i for i in argv]
    dir_name = "/content/gdrive/"+ PATH + "pickles/"
    filename_suffix = "pkl"
    file_path = os.path.join(dir_name, filename + "." + filename_suffix)
    with open(file_path,'rb') as f:
        variable_list = pickle.load(f)
    return variable_list

In [ ]:
tr_cat,te_cat= read_pickle_dump('tr_cat','te_cat',filename = "cat_features")

### **3.2 Text feature vectorizations:**

**TFIDF Vectorizer for concatenated text:**

Các đoạn văn bản sẽ được encoded sang dạng vectors, tại đây sử dụng TFIDFVectorizer cho item_description và BoW cho name

**BoW for name**

In [ ]:
# from nltk.corpus import stopwords 
# from nltk.tokenize import word_tokenize

In [ ]:
# # hàm này chưa test và cần chuyển đổi về dạng list về chuỗi
# def remove_stop_words(data):
#     ser = pd.Series()

#     fox i in data:
#         stop_words = set(stopwords.words('english')) 
#         word_tokens = word_tokenize(i) 
#         filtered_sentence = [w for w in word_tokens if not w in stop_words] 
#         filtered_sentence = [] 
            
#         for w in word_tokens: 
#             if w not in stop_words: 
#                 filtered_sentence.append(w)
#         ser.append(filtered_sentence)
#     return ser
    

In [ ]:
def vectorizer(train,test,column,no_of_features,n_range,vector_type):
    '''
    Function to vectorize text using TFIDF/BoW
    '''
    if str(vector_type) == 'bow':
        vectorizer = CountVectorizer(ngram_range=n_range,max_features=no_of_features).fit(train[column])  #fitting
    else:
        vectorizer = TfidfVectorizer(ngram_range=n_range, max_features=no_of_features).fit(train[column]) # fit has to happen only on train data
    
    # we use the fitted vectorizer to convert the text to vector
    transformed_text = vectorizer.transform(tqdm(test[column]))
    print("After vectorizations")
    print(transformed_text.shape)
    print("="*100)
    return transformed_text

In [ ]:
X_train_bow_name = vectorizer(X_train,X_train,'name',100000,(1,2),'bow')
X_test_bow_name = vectorizer(X_train,X_test,'name',100000,(1,2),'bow')

100%|██████████| 1333494/1333494 [00:17<00:00, 75317.33it/s]


After vectorizations
(1333494, 100000)


100%|██████████| 148167/148167 [00:01<00:00, 76843.38it/s]


After vectorizations
(148167, 100000)


**TFIDF vectorization for item_description:**

In [ ]:
X_train_tfidf_desc = vectorizer(X_train,X_train,'item_description',100000,(1,2),'tfidf')
X_test_tfidf_desc = vectorizer(X_train,X_test,'item_description',100000,(1,2),'tfidf')

100%|██████████| 1333494/1333494 [01:15<00:00, 17546.15it/s]


After vectorizations
(1333494, 100000)


100%|██████████| 148167/148167 [00:08<00:00, 17956.49it/s]


After vectorizations
(148167, 100000)


In [ ]:
pickle_dump(X_train_tfidf_desc,X_test_tfidf_desc,filename="tfidf_desc")

In [ ]:
X_train_tfidf_desc,X_test_tfidf_desc = read_pickle_dump('X_train_tfidf_desc','X_test_tfidf_desc',filename="tfidf_desc")

### **3.3 Feature matrix**


**Feature matrix:**

BoW of name + TFIDF of item_description + OHEs of categorical columns + item_id and shipping dummies + wordcount of name + wordcount of description text

In [ ]:
x_train_set = hstack((X_train_bow_name,X_train_tfidf_desc,tr_cat,X_train['wc_name'].values.reshape(-1,1),X_train['wc_desc'].values.reshape(-1,1))).tocsr()
x_test_set = hstack((X_test_bow_name,X_test_tfidf_desc,te_cat,X_test['wc_name'].values.reshape(-1,1),X_test['wc_desc'].values.reshape(-1,1))).tocsr()

print("Final Data matrix")
print(x_train_set.shape)
print(x_test_set.shape)

Final Data matrix
(1333494, 206949)
(148167, 206949)


chuyển đổi giá của sản phẩm sang dạng log 

In [ ]:
def log_price(price):
    return np.log1p(price)

In [ ]:
global y_scalar
y_scalar = StandardScaler()
y_train = y_scalar.fit_transform(log_price(X_train['price']).values.reshape(-1, 1))
y_test = y_scalar.transform(log_price(X_test['price']).values.reshape(-1, 1))